In [ ]:
#Reloads the lab.py and crystals.py modules to update any changes (after saving)
#If a new method or object is created, autoreload doesn't work and the 
#kernel needs to be closed and halted after saving and making a 'checkpoint'
#in this notebook

%load_ext autoreload
%autoreload 2

In [ ]:
import ipas 
import numpy as np
import dask
from dask_jobqueue import SLURMCluster
from distributed import LocalCluster
from dask.distributed import Client, progress
from dask import delayed
from dask import dataframe as dd
import functools
import sys
import ast
from struct import *
import pickle
import glob
import random
import pandas as pd
import time
from dask.distributed import as_completed
from joblib import Parallel, delayed, parallel_backend
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors

In [ ]:
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (7,7),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)

In [ ]:
cluster = SLURMCluster(
    queue='batch',
    walltime='04-23:00:00',
    cores=1,
    memory='20000MiB', #1 GiB = 1,024 MiB
    processes=1)

#cluster.adapt(minimum=3, maximum=20)
cluster.scale(1)

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
def shape(a,b,c):
    if (b-c) <= (a-b):
        return 'prolate'
    else:
        return 'oblate'

In [ ]:
%%time 
files = [f for f in glob.glob("../instance_files/createdb_iceagg_flat*")]
data = []
for file in files:
    print(file)
    #dictionary = pickle.load(f)
    data.append(pd.read_pickle(file, None))
datapd = [pd.DataFrame(i) for i in data]
df = pd.concat(datapd, axis=0, ignore_index=True)

In [ ]:
#speed up shape function 
vfunc = np.vectorize(shape)
df['shape'] = vfunc(df['a'], df['b'], df['c'])
df['agg_phi'] = df.c/df.a
df.loc[df['shape'] == 'oblate', 'agg_r'] = np.power((np.power(df['a'], 2) * df['c']), (1./3.))
df.loc[df['shape'] == 'prolate', 'agg_r'] = np.power((np.power(df['c'], 2) * df['a']), (1./3.))
#df['agg_r'] = np.power((np.power(df['a'], 2) * df['c']), (1./3.))
df = df[df.agg_r < 5000]


# MAIN

In [ ]:
def main(agg_phi_bins, agg_r_bins, nclusters, rand_orient, phi_factor=10, r_factor=2):
    
    output = np.empty((agg_phi_bins,agg_r_bins),dtype=object)
    hold_clusters  = np.empty((agg_phi_bins,agg_r_bins,nclusters), dtype=object)
    a  = np.empty((agg_phi_bins,agg_r_bins,nclusters), dtype=object)
    c = np.empty((agg_phi_bins,agg_r_bins,nclusters), dtype=object)
    
    count_mono_phi_lower=0
    count_mono_phi_upper=0
    count_mono_r_lower=0
    count_mono_r_upper=0
    
    res, phi_bins = pd.qcut(df.agg_phi, agg_phi_bins, retbins=True)
    #print(phi_bins)
    for i in range(0, agg_phi_bins):
        #print('agg phi range: ', phi_bins[i], phi_bins[i+1])
        #return a df that only queries within an aspect ratio bin
        df_phi = df[(df.agg_phi > phi_bins[i]) & (df.agg_phi < phi_bins[i+1])]  
        #to ensure at least 2 crystals within agg since ncrystals=1 not in db
        #now break that aspect ratio bin into 20 equal r bins
        
        res, r_bins = pd.qcut(df_phi.agg_r, agg_r_bins, retbins=True)
        for r in range(0, agg_r_bins):   #agg r
               
            #print('r = ', r_bins[r], r_bins[r+1])
            df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1])]
            #plt.hist(df_r.mono_phi)
            #plt.xscale('log')
            #plt.show()

            samples = df_r.sample(nclusters)
            for n, agg in enumerate(samples.itertuples()):

                phi_range = np.logspace(np.log(agg.mono_phi/phi_factor)/np.log(10),
                                        np.log(agg.mono_phi*phi_factor)/np.log(10), 20)
                mono_phi = random.choice(phi_range)
                #mono_phi = 0.01 if mono_phi < 0.01 else mono_phi
                #mono_phi = 70 if mono_phi > 70 else mono_phi
                if mono_phi < 0.01:
                    count_mono_phi_lower+=1
                    mono_phi=0.01
                if mono_phi > 70.:
                    count_mono_phi_upper+=1
                    mono_phi=70.

                r_range = np.logspace(np.log(agg.mono_r/r_factor)/np.log(10),\
                                      np.log(agg.mono_r*r_factor)/np.log(10),20)
                mono_r = random.choice(r_range)
                #mono_r = 1000 if mono_r>1000 else mono_r
                #mono_r = 1 if mono_r<1 else mono_r
                if mono_r < 1.0:
                    count_mono_r_lower+=1
                    mono_r = 1.0
                if mono_r > 1000:
                    count_mono_phi_upper+=1
                    mono_phi = 1000
                
                a[i,r,n] = (mono_r ** 3 / mono_phi) ** (1. / 3.)
                c[i,r,n] = mono_phi * a[i,r,n]
#               print('phi range', agg.mono_phi, phi_range[0], phi_range[-1], phi_range)
#               print('r range: ', agg.mono_r, r_range[0], r_range[-1], r_range)
                
                hold_clusters[i,r,n] = ipas.Cluster_Calculations(agg)

            #ipas.collect_clusters(a[i,r,:], c[i,r,:], hold_clusters[i,r,:], rand_orient=rand_orient)
            #output[i,r] = dask.delayed(ipas.collect_clusters)(a[i,r,:], c[i,r,:],
            #                                                     hold_clusters[i,r,:], rand_orient=rand_orient)
    print(count_mono_phi_lower/120000,\
          count_mono_phi_upper/120000,\
          count_mono_r_lower/120000,\
          count_mono_r_upper/120000)
    return output, hold_clusters
    

In [ ]:
def compute():
    agg_as = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    agg_bs = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    rzs = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    phi2Ds = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    cplxs = np.empty((agg_phi_bins, agg_r_bins, nclusters))
    dds = np.empty((agg_phi_bins, agg_r_bins, nclusters))

    gather = client.compute([*output.tolist()]) 
    gather = client.gather(gather)
    gather = np.array(gather)
    print(np.shape(gather))
    agg_as = gather[:,:,0,:]
    agg_bs = gather[:,:,1,:]
    agg_cs = gather[:,:,2,:]
    phi2Ds = gather[:,:,3,:]
    cplxs = gather[:,:,4,:] 
    dds = gather[:,:,5,:]

    print('DONE!')
    return agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds

In [ ]:
if __name__ == '__main__':
    
    agg_phi_bins = 20
    agg_r_bins = 20
    nclusters = 300
    rand_orient = False

    output, hold_clusters = main(agg_phi_bins, agg_r_bins, nclusters, rand_orient)
    agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds= compute()
    results = {'agg_as': agg_as, 'agg_bs':agg_bs, 'agg_cs':agg_cs, 'phi2Ds':phi2Ds, \
               'cplxs':cplxs, 'dds':dds}
   
#     filename = '../instance_files/pulled_clusters_iceagg_flat'
#     filehandler = open(filename, 'wb')
#     pickle.dump(hold_clusters, filehandler)
#     filehandler.close()
    
#     filename = '../instance_files/instance_db_iceagg_flat'
#     filehandler = open(filename, 'wb')
#     pickle.dump(results, filehandler)
#     filehandler.close()
    print('finished!')

## Make bin plots

In [ ]:
def query_ncrystals(df_phi, r_bins):
    avg_ncrystals = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1])]
        avg_ncrystals.append(df_r.ncrystals.mean())
    return avg_ncrystals


In [ ]:
def avg_cplx(df_phi, r_bins):
    avg_cplx = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.cplx > r_bins[r]) & (df_phi.cplx < r_bins[r+1])]
        avg_cplx.append(np.mean(df_r.cplx))
    return avg_cplx


In [ ]:
def oblate_prolate(df_phi, r_bins):
    shape = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1]) & (df_phi.ncrystals > 2)]
        oblates = df_r['shape'][df_r['shape'] == 'oblate'].count()
        prolates = df_r['shape'][df_r['shape'] == 'prolate'].count()
        shape.append(oblates-prolates)
        return shape

In [ ]:
def plate_columns_agg(df_phi, r_bins):
    agg_mono_phi = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1]) & (df_phi.ncrystals > 2)]
        agg_mono_plates = df_r['mono_phi'][df_r['mono_phi'] < 1.0].count()
        agg_mono_col = df_r['mono_phi'][df_r['mono_phi'] > 1.0].count()
        agg_mono_phi.append(agg_mono_plates - agg_mono_col)
    return agg_mono_phi


In [ ]:
def average_radius(df_phi, r_bins):
    avg_radius = []
    for r in range(len(r_bins)-1):
        df_r = df_phi[(df_phi.agg_r > r_bins[r]) & (df_phi.agg_r < r_bins[r+1])]
        avg_radius.append(np.mean(df_r.mono_r))
    return avg_radius

In [ ]:
#RAND
res, phi_bins_rand = pd.qcut(df.agg_phi, 20, retbins=True)
#print(phi_bins)
phi_bin_labs = []
avg_ncrystals=np.empty((len(phi_bins_rand)-1,len(phi_bins_rand)-1))
avg_cplxs=np.empty((len(phi_bins_rand)-1,len(phi_bins_rand)-1))
all_r_bins= np.empty((len(phi_bins_rand),len(phi_bins_rand)))
shape= np.empty((len(phi_bins_rand)-1,len(phi_bins_rand)-1))
agg_mono_phi = np.empty((len(phi_bins_rand)-1,len(phi_bins_rand)-1))
avg_radius = np.empty((len(phi_bins_rand)-1,len(phi_bins_rand)-1))

for i in range(agg_phi_bins):
    print('i = ', i)
    phi_bin_labs.append('[%.3f-%.3f]' %(phi_bins_rand[i],phi_bins_rand[i+1]))
    #return a df that only queries within an aspect ratio bin
    df_phi = df[(df.agg_phi > phi_bins_rand[i]) & (df.agg_phi < phi_bins_rand[i+1])]
    #now break that aspect ratio bin into 20 equal r bins
    res, r_bins = pd.qcut(df_phi.agg_r, 20, retbins=True)
    all_r_bins[i,:] = r_bins
    
    #now use those r bins from the output of queried r and phi to find # of monomers per bin
    avg_ncrystals[i,:] = query_ncrystals(df_phi, r_bins)
    avg_cplxs[i,:] = avg_cplx(df_phi, r_bins)
    shape[i,:] = oblate_prolate(df_phi, r_bins)
    agg_mono_phi[i,:] = plate_columns_agg(df_phi, r_bins)
    avg_radius[i,:] = average_radius(df_phi, r_bins)
    

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
cmap = plt.cm.seismic
cb_range = max(abs(np.amin(avg_ncrystals)), np.amax(avg_ncrystals))
divnorm = colors.DivergingNorm(vmin=-cb_range, vcenter=0, vmax=cb_range)

#norm = matplotlib.colors.Normalize(vmin=np.amin(shape), vmax=np.amax(shape))
for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):

        if r != 0:
            if shape[i,r] == 'p': 
                plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(divnorm(avg_ncrystals[i,r])),edgecolor='k')
            else:
                 plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(divnorm(avg_ncrystals[i,r])),edgecolor='k')
        else:
            if shape[i,r] == 'p':
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(divnorm(avg_ncrystals[i,r])), edgecolor='k')
            else:
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(divnorm(avg_ncrystals[i,r])), edgecolor='k')
                
    
plt.yscale('log')
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center",fontsize=16,family='serif')
plt.ylabel("Aggregate Radius Bins",fontsize=16,family='serif')
plt.xlabel("Aggregate Aspect Ratio ($\phi$) bins",fontsize=16,family='serif')  
cb = plt.cm.ScalarMappable(norm=divnorm, cmap=cmap)
cbar = plt.colorbar(cb)
#cbar.ax.set_ylabel('Average # of monomers per bin', fontsize=16, family='serif')
#cbar.ax.set_ylabel('Average # of monomers per bin', fontsize=16, family='serif')
#plt.title('Quasi-Horizontal Orientation',fontsize=16, family='serif')
plt.title('Flat Orientation',fontsize=16, family='serif')
plt.tight_layout()
#plt.savefig('bins_rand_meanmono_r_5000rad_logy.pdf')


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
cmap = plt.cm.jet
variable = avg_radius
norm = matplotlib.colors.LogNorm(vmin=np.amin(variable), vmax=np.amax(variable))

#norm = matplotlib.colors.Normalize(vmin=np.amin(shape), vmax=np.amax(shape))
for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):

        if r != 0:
            if shape[i,r] == 'p': 
                plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
            else:
                 plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
        else:
            if shape[i,r] == 'p':
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')
            else:
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')
                
    
plt.yscale('log')
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center",fontsize=16,family='serif')
plt.ylabel("Aggregate Radius Bins",fontsize=16,family='serif')
plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins",fontsize=16,family='serif')  
cb = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = plt.colorbar(cb)
cbar.ax.set_ylabel('Average Monomer Radius per Bin', fontsize=16, family='serif')
#plt.title('Quasi-Horizontal Orientation',fontsize=16, family='serif')
plt.title('Flat Orientation',fontsize=16, family='serif')
plt.tight_layout()
#plt.savefig('bins_rand_meanmono_r_5000rad_logy.pdf')


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
cmap = plt.cm.jet
variable = avg_ncrystals
norm = matplotlib.colors.Normalize(vmin=2, vmax=25)

for i in range(agg_phi_bins): 
    print('i= ', i)
    for r in range(agg_r_bins):

        if r != 0:
            if shape[i,r] == 'p': 
                plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
            else:
                 plt.bar([i]*len(r_bins), all_r_bins[i,r], bottom= all_r_bins[i,r-1],  color=cmap(norm(variable[i,r])),edgecolor='k')
        else:
            if shape[i,r] == 'p':
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')
            else:
                plt.bar([i]*len(r_bins), all_r_bins[i,r], color=cmap(norm(variable[i,r])), edgecolor='k')
                
    
plt.yscale('log')
plt.xticks(np.arange(len(phi_bin_labs)), phi_bin_labs, rotation=90, ha="center",fontsize=16,family='serif')
plt.ylabel("Aggregate Radius Bins",fontsize=16,family='serif')
plt.xlabel("Aggregate Aspect Ratio ($\phi$) Bins",fontsize=16,family='serif')  
cb = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
cbar = plt.colorbar(cb, format='%d')
cbar.ax.set_ylabel('Average # of Monomers per Bin', fontsize=16, family='serif')
#plt.title('Quasi-Horizontal Orientation',fontsize=16, family='serif')
plt.title('Flat Orientation',fontsize=16, family='serif')
plt.tight_layout()
#plt.savefig('bins_rand_meanmono_r_5000rad_logy.pdf')


In [ ]:
f = open('pulled_clusters_aggagg_flat', 'rb')
clus = pickle.load(f)
f.close()

In [ ]:
f = open('../instance_files/pulled_clusters_aggagg_flat', 'rb')
clusters = pickle.load(f)
f.close()
